In [10]:
# check headers
import csv

def get_headers(in_file):
    with open(in_file, encoding = 'utf-8-sig', errors = 'ignore', newline = '') as file_in:
        reader = csv.reader(file_in)
        rowcount = 0
        endrow = 'NOTSET'
        headers = {}
        for row in reader:
            if rowcount == 0:
                for cell in row:
                    if cell != '':
                        header = {row.index(cell): cell}
                        headers.update(header)
                    elif cell == '':
                        endrow = row.index(cell)
                        pass
                    else:
                        pass
                rowcount += 1
            else:
                pass
    print(endrow)
    print(type(endrow))
    print(headers)

get_headers('testdata/csv_test_in_001.csv')
get_headers('testdata/csv_test_in_002.csv')


NOTSET
<class 'str'>
{0: 'cdmnumber', 1: 'string', 2: 'attribute', 3: 'value', 4: 'comment'}
58
<class 'int'>
{0: 'cdmnumber', 1: 'Title', 2: 'Title (Translated)', 3: 'Title (Non-Latin Script)', 4: 'Title (ALA-LC Romanization)', 5: 'Title (Harvard Romanization)', 6: 'Interviewee', 7: 'Interviewee Birthplace', 8: 'Primary Performer', 9: 'Performer', 10: 'Performer Birthplace', 11: 'Interviewer', 12: 'Videographer', 13: 'Author', 14: 'Editor', 15: 'Publisher', 16: 'Publisher Location', 17: 'Printer', 18: 'Printer Location', 19: 'Translator', 20: 'Donor', 21: 'Lender', 22: 'Former Owner', 23: 'Date (EDTF)', 24: 'Date', 25: 'Dates', 26: 'Interviewee Biography', 27: 'Notes', 28: 'Subjects (LCSH)', 29: 'Genre/Form (LCGFT)', 30: 'Related OCLC Record', 31: 'Object Type', 32: 'Cross Reference', 33: 'Romancero Theme', 34: 'Individuals Discussed', 35: 'Organizations Discussed', 36: 'Geographic Coverage', 37: 'Time Period Discussed', 38: 'Index', 39: 'Recording Location', 40: 'Language', 41: 'Digi

# This (^^^) is interesting
behavior is different for fewer columns vs. less

# the current draft, the current problem...
...getting a KeyError with [this version](https://gist.github.com/briesenberg07/1d3bdc9d079a8581768088dea1111f6b/4ff203a01d6b520bc5c444b00430ef80688825aa#file-_reformat_csv_001-py)...

In [9]:
# ...
import csv

def convert(file_in, file_out):
    with open(file_in, encoding = 'utf-8-sig', errors = 'ignore', newline = '') as file_in:
        reader = csv.reader(file_in)
        rowcount = 0
        emptyrows = 0
        headers = {}

        with open(file_out, encoding = 'utf-8-sig', mode = 'w', newline = '') as file_out:
            writer = csv.writer(file_out, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
            newheaders = ['cdmnumber', 'value', 'field']
            writer.writerow(newheaders)
            for row in reader:
                if rowcount == 0:
                    for cell in row:
                        header = {row.index(cell): cell}
                        headers.update(header)
                        rowcount += 1
                elif rowcount > 0 and row[0] != '':
                    for cell in row:
                        newrow = []
                        if row.index(cell) == 0:
                            pass
                        else:
                            newrow.append(row[0])
                            newrow.append(cell)
                            newrow.append(headers[row.index(cell)]) # getting errors here!
                            writer.writerow(newrow)
                    rowcount += 1
                elif rowcount > 0 and row[0] == '':
                    emptyrows += 1
                else:
                    pass # not quite sure how an else could occur...
    # very minimal report
    # add better printout of headers
    print(headers)
    print(f"{rowcount} rows containing data were processed")
    if emptyrows == 1:
        print(f"{emptyrows} empty row was skipped")
    elif emptyrows > 1:
        print(f"{emptyrows} empty rows were skipped")
    else:
        pass

file_in = input("Enter filepath for the input CSV file: ")
file_out = input("Enter filepath for an output CSV file to write: ")
convert(file_in, file_out)

Enter filepath for the input CSV file: testdata/csv_test_in_002.csv
Enter filepath for an output CSV file to write: testdata/csv_test_out_002.csv


KeyError: 58

## KeyError!!
```
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-9-6e1e6a594012> in <module>
     50 file_in = input("Enter filepath for the input CSV file: ")
     51 file_out = input("Enter filepath for an output CSV file to write: ")
---> 52 convert(file_in, file_out)

<ipython-input-9-6e1e6a594012> in convert(file_in, file_out)
     30                             newrow.append(row[0])
     31                             newrow.append(cell)
---> 32                             newrow.append(headers[row.index(cell)]) # getting errors here!
     33                             writer.writerow(newrow)
     34                     rowcount += 1

KeyError: 58
```
### I THINK THAT...
...I need to stop the script from attempting to process the cell following the last cell which has a value in it...
...so perhaps grabbing the index of the last column with a header in it, and using that to limit processing...
maybe something like:

```
[get an index of the first empty column after the headers]
...
for cell in row:
    if row.index(cell) < maxIndex:
        ...
```


In [14]:
# ...
import csv

def convert(file_in, file_out):
    with open(file_in, encoding = 'utf-8-sig', errors = 'ignore', newline = '') as file_in:
        reader = csv.reader(file_in)
        rowcount = 0
        emptyrows = 0
        headers = {}
        endheaders = 'NOTSET'

        with open(file_out, encoding = 'utf-8-sig', mode = 'w', newline = '') as file_out:
            writer = csv.writer(file_out, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
            newheaders = ['cdmnumber', 'value', 'field']
            writer.writerow(newheaders)
            for row in reader:
                if rowcount == 0:
                    for cell in row:
                        if cell != '':
                            header = {row.index(cell): cell}
                            headers.update(header)
                            rowcount += 1
                        else:
                            endheaders = row.index(cell)
                elif rowcount > 0 and row[0] != '':
                    for cell in row:
                        newrow = []
                        if row.index(cell) == 0:
                            pass
                        elif row.index(cell) > 0 and row.index(cell) < endheaders:
                            newrow.append(row[0])
                            newrow.append(cell)
                            newrow.append(headers[row.index(cell)]) # getting errors here!
                            writer.writerow(newrow)
                        else:
                            pass
                    rowcount += 1
                elif rowcount > 0 and row[0] == '':
                    emptyrows += 1
                else:
                    pass # not quite sure how an else could occur...
    # improve report
    print(headers)
    print(f"{rowcount} rows containing data were processed")
    if emptyrows == 1:
        print(f"{emptyrows} empty row was skipped")
    elif emptyrows > 1:
        print(f"{emptyrows} empty rows were skipped")
    else:
        pass

file_in = input("Enter filepath for the input CSV file: ")
file_out = input("Enter filepath for an output CSV file to write: ")
convert(file_in, file_out)


Enter filepath for the input CSV file: testdata/csv_test_in_002.csv
Enter filepath for an output CSV file to write: testdata/csv_test_out_002.csv
{0: 'cdmnumber', 1: 'Title', 2: 'Title (Translated)', 3: 'Title (Non-Latin Script)', 4: 'Title (ALA-LC Romanization)', 5: 'Title (Harvard Romanization)', 6: 'Interviewee', 7: 'Interviewee Birthplace', 8: 'Primary Performer', 9: 'Performer', 10: 'Performer Birthplace', 11: 'Interviewer', 12: 'Videographer', 13: 'Author', 14: 'Editor', 15: 'Publisher', 16: 'Publisher Location', 17: 'Printer', 18: 'Printer Location', 19: 'Translator', 20: 'Donor', 21: 'Lender', 22: 'Former Owner', 23: 'Date (EDTF)', 24: 'Date', 25: 'Dates', 26: 'Interviewee Biography', 27: 'Notes', 28: 'Subjects (LCSH)', 29: 'Genre/Form (LCGFT)', 30: 'Related OCLC Record', 31: 'Object Type', 32: 'Cross Reference', 33: 'Romancero Theme', 34: 'Individuals Discussed', 35: 'Organizations Discussed', 36: 'Geographic Coverage', 37: 'Time Period Discussed', 38: 'Index', 39: 'Recording 

# &#x1F389; &#x1F389; &#x1F389; ???
## KeyError solved
...on to the next problem